In [28]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import datetime as dtm
import matplotlib.dates as mpd
import pytz
import calendar
import operator
import sys
#
# TODO: it is probably time to say goodnight to Python 2.x support...
import urllib
# try:
# 	# should work with python 3.x
# 	import urllib.request, urllib.parse, urllib.error
# except:
# 	print("failed while loading: urllib.request, urllib.parse, urllib.error.\n probably Python 2.x?")
# 	#urllib.request.urlopen = urllib.urlopen

# try:
# 	import ullib2
# except:
# 	print("failed while loading urllib and/or urllib. maybe python 3.x?")
	
#import urllib.request, urllib.error, urllib.parse
import requests
import numpy
import math

tzutc = pytz.timezone('UTC')

### ComCat ANSS API:
- Comcat is awesome, and it replaced the old ANSS API (that was basicaly a hack I did)
- But it's bulky and difficult to install on HPC or managed systems
- So there's a new API, which i think is even easier to use. we should reserve it as an option
- ... and develop it here.

In [85]:
#TODO: 
# maybe move this to ANSS_tools, and develop on a branch like a grownup?
#
# here is the new ANSS-comcat portal:
# https://earthquake.usgs.gov/earthquakes/search/
#
# and here is a sample results URL:
# https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=2019-09-06%2000:00:00&endtime=2019-09-13%2023:59:59&maxlatitude=37&minlatitude=30&maxlongitude=-115&minlongitude=-122&minmagnitude=2.5&eventtype=earthquake&orderby=time
#
# it looks like we can just replace the URL and use a query string, rather than post syntax, and then emulate
#. the remaining infrastructure. in fact, the hierarchal get_list() -> process_list_for_catalog() organization
#. might not look so silly any longer, since we presumably only have to rewrite the very top layer.
#
#
# copy some stuff from ANSS tools; we'll code this up here and then move it over:

def anss_comcat_DateStr(x=dtm.datetime.now(pytz.timezone('UTC')), delim_dt='-', delim_tm=':', dt_tm_sep=' '):
    # yoder, 13 july 2015: ANSS seems to have made some changes. these date formats are breaking. probalby a matter of leading 0's in dates; might be fractional seconds.
    #yr=x.year
    #mo=x.month
    #dy=x.day
    #hr=x.hour
    #mn=x.minute
    #sc=x.second
    #ms=x.microsecond
    #fsecs=float(sc) + float(ms)*(10**(-6.0))
    #
    yr = str(x.year)
    mo = ('00' + str(x.month))[-2:]
    dy = ('00' + str(x.day))[-2:]
    hr = ('00' + str(x.hour))[-2:]
    mn = ('00' + str(x.minute))[-2:]
    sc = ('00' + str(x.second))[-2:]
    #
    # ANSS seems to be complaining about fractional seconds, so skip this and return integer seconds.
    '''
    ms=x.microsecond
    fsecs=float(sc) + float(ms)*(10**(-6.0))
    #
    # trim extra zeros:
    fsecs_str = str(fsecs)
    while ('.' in fsecs_str and len(fsecs_str)>3 and fsecs_str[-1]=='0'):
        fsecs_str = fsecs_str[:-1]
    '''
    #
    #return '%s/%s/%s,%s:%s:%f' % (yr, mo, dy, hr, mn, fsecs)
    #return '%s/%s/%s,%s:%s:%s' % (yr, mo, dy, hr, mn, sc)

    return delim_dt.join([yr,mo,dy]) + dt_tm_sep + delim_tm.join([hr,mn,sc])
    #
# let's take this opportunity to revise our syntax and introduce class structure. We can maintain backwards
# compatibility with a function wrapper.
class ANSS_Comcat_catalog(object):
    # TODO: datestring re-formatting is totally forked. need to figure that out...
    #
    anss_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query.csv'
    
    def __init__(self, min_lon=-125., max_lon=-115., min_lat=32., max_lat=45., m_c=3.5,
                 from_date=dtm.datetime(2000, 1,1, tzinfo=tzutc), to_date=dtm.datetime.now(tzutc),
                 Nmax=None):
        #
        delim_dt = '-'
        delim_tm = ':'
        #
        if to_date is None:
            to_date = dtm.datetime.now(tzutc)
        from_date = anss_comcat_DateStr(from_date, delim_dt=delim_dt, delim_tm=delim_tm, dt_tm_sep='%20')
        to_date   = anss_comcat_DateStr(to_date, delim_dt=delim_dt, delim_tm=delim_tm, dt_tm_sep='%20')
        #
        #print('*** DEBUG: from_date:: {}'.format(from_date))
        #print('*** DEBUT: to_date:: {}'.format(to_date))
        #
        # 'starttime':from_date, 'endtime':to_date,
        anssPrams={  'minmagnitude':m_c, 'minlatitude':min_lat, 'maxlatitude':max_lat, 'minlongitude':min_lon, 
                   'maxlongitude':max_lon, 
                   'eventtype':'earthquake', 'orderby':'time', 'limit':Nmax
                  }
        anss_prams = {ky:vl for ky,vl in anssPrams.items() if not (vl in (chr(9), chr(32)) or vl is None)}
        #
        #
        url_str = '{}?starttime={}&endtime={}&{}'.format(self.anss_url,from_date, to_date, 
                                                      urllib.parse.urlencode(anss_prams) )
        # = 'https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=2019-09-01%2000:00:00&endtime=2019-09-14%2006:16:43&limit=500&minmagnitude=3.5&minlatitude=32.0&maxlatitude=45.0&minlongitude=-125.0&maxlongitude=-115.0&eventtype=earthquake&orderby=time'
        #print('*** DEBUG:  ', url_str)
        #f = urllib.request.urlretrieve(url_str)
        #
        self.__dict__.update({ky:val for ky,val in locals().items() if not ky in ('self', '__class__')})
    #
    @property
    def f(self):
        return self.get_f()
    #
    def get_f(self):
        return urllib.request.urlopen(self.url_str)

In [86]:
obj =  ANSS_Comcat_catalog(from_date=dtm.datetime(2019, 9, 1))

for rw in obj.f:
    print('*** ', rw)

print('** ', obj.f)
obj.f.close()
#    print('** ', rw)

***  b'time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\n'
***  b'2019-09-13T06:50:21.890Z,36.6031667,-121.202,6.03,3.82,mw,61,66,0.03413,0.09,nc,nc73271485,2019-09-14T05:05:46.011Z,"10km NNW of Pinnacles, CA",earthquake,0.15,0.43,,5,reviewed,nc,nc\n'
***  b'2019-09-10T20:21:49.840Z,33.594,-117.27,13.69,3.96,ml,178,21,0.06253,0.23,ci,ci38824959,2019-09-14T05:15:51.962Z,"1km ESE of Wildomar, CA",earthquake,0.12,0.29,0.151,333,reviewed,ci,ci\n'
***  b'2019-09-10T12:24:42.590Z,39.5272,-116.4999,9.4,3.8,ml,44,85.35,0.872,0.2833,nn,nn00703221,2019-09-11T12:25:50.008Z,"46km E of Austin, Nevada",earthquake,,2.7,0.21,17,reviewed,nn,nn\n'
***  b'2019-09-10T04:25:55.631Z,39.5419,-116.5599,9.1,4.6,ml,53,67.17,0.857,0.2275,nn,nn00703188,2019-09-11T04:27:01.615Z,"45km E of Austin, Nevada",earthquake,,2.2,0.27,23,reviewed,nn,nn\n'
***  b'2019-09-08T14:07:23.340Z,35.5815,-117.4133333,5.57,3.

In [25]:
#
def getANSStoFilehandler(lon=[-125, -115], lat=[32, 45], minMag=4.92, dates0=[dtm.datetime(2001,1,1, tzinfo=tzutc), dtm.datetime(2010, 12, 31, tzinfo=tzutc)], Nmax=999999):
    # fetch data from ANSS; return a file handler.
    #
    # use urllib in "post" mode. an example from http://www.python.org/doc/current/library/urllib.html#urllib.FancyURLopener)
    # using "get" (aka, query-string method; note the ?%s string at the end of the URL, this is a single pram call to .urlopen):
    #
    #>>> import urllib
    #>>> params = urllib.urlencode({'spam': 1, 'eggs': 2, 'bacon': 0})
    #>>> f = urllib.urlopen("http://www.musi-cal.com/cgi-bin/query?%s" % params)
    #>>> print f.read()
    #
    # using "post" (note this is a 2 pram call):
    #>>> import urllib
    #>>> params = urllib.urlencode({'spam': 1, 'eggs': 2, 'bacon': 0})
    #>>> f = urllib.urlopen("http://www.musi-cal.com/cgi-bin/query", params)
    #>>> print f.read()
    #
    # make ANSS prams dictionary (thank james for the bash-template):
    # ANSSquery has day-resolution:
    # revision: ANSS has time resolution, but you have to replace "-" -> "/" and the " " (space) -> ","
    #dates=[dtm.date(dates0[0].year, dates0[0].month, dates0[0].day), dtm.date(dates0[1].year, dates0[1].month, dates0[1].day)]
    dates=dates0
    datestr1 = anssDateStr(dates[0])
    datestr2 = anssDateStr(dates[1])
    #print datestr1, datestr2
    #
    #anssPrams={'format':'cnss', 'output':'readable', 'mintime':str(dates[0]).replace('-', '/'), 'maxtime':str(dates[1]).replace('-', '/'), 'minmag':str(minMag), 'minlat':lat[0], 'maxlat':lat[1], 'minlon':lon[0], 'maxlon':lon[1], 'etype':'E', 'searchlimit':Nmax}
    # so this is better, but i think it is still limited to 1 second resolution.
    #
    anssPrams={'format':'cnss', 'output':'readable', 'mintime':datestr1, 'maxtime':datestr2, 'minmag':str(minMag), 'minlat':lat[0], 'maxlat':lat[1], 'minlon':lon[0], 'maxlon':lon[1], 'etype':b'E', 'searchlimit':Nmax}
    #anssPrams={'format':b'cnss', 'output':b'readable', 'mintime':bytearray(datestr1, 'utf-8'), 'maxtime':bytearray(datestr2, 'utf-8'), 'minmag':bytearray(str(minMag), 'utf-8'), 'minlat':lat[0], 'maxlat':lat[1], 'minlon':lon[0], 'maxlon':lon[1], 'etype':b'E', 'searchlimit':Nmax}
    #print "debug: ", anssPrams
    #post_data = urllib.parse.urlencode(anssPrams)
    #binary_post_data = post_data.encode('ascii')
    #
    # now, let's support some backwards compatibility, at least for a little while:
    if sys.version_info.major == 2:
        # old python...
        f = urllib.urlopen('http://www.ncedc.org/cgi-bin/catalog-search2.pl', urllib.urlencode(anssPrams))
    else:
        binary_post_data = urllib.parse.urlencode(anssPrams).encode('ascii')
        f = urllib.request.urlopen('http://www.ncedc.org/cgi-bin/catalog-search2.pl', binary_post_data )
    #
    # we might return f, a string of f, or maybe a list of lines from f. we'll work that out shortly...
    return f

In [18]:
import urllib.urlretrieve

ModuleNotFoundError: No module named 'urllib.urlretrieve'

In [ ]:
https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=2019-09-06%2000:00:00&endtime=2019-09-13%2023:59:59&
        maxlatitude=35&minlatitude=31&maxlongitude=-113&minlongitude=-118&minmagnitude=2.5&maxmagnitude=9&
        mindepth=-1&maxdepth=300&eventtype=earthquake&orderby=time&limit=500
        

https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=2019-09-01%2000:00:00&endtime=2019-09-14%2006:16:43&
                    minmagnitude=3.5&minlatitude=32.0&maxlatitude=45.0&minlongitude=-125.0&maxlongitude=-115.0&
                    eventtype=earthquake&orderby=time&limit=500
                    
https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=2019-09-01%2000:00:00&endtime=2019-09-14%2006:16:43&limit=500&minmagnitude=3.5
                    
                    